In [ ]:
from pyincore import IncoreClient, FragilityService, MappingSet
from pyincore.analyses.montecarlofailureprobability import MonteCarloFailureProbability
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.waterfacilitydamage import WaterFacilityDamage

## Monte Carlo failure probability analysis

In [ ]:
client = IncoreClient()

### Chaining with Building damage analysis

In [ ]:
bldg_dmg = BuildingDamage(client)
# Memphis hospitals
bldg_dmg.load_remote_input_dataset("buildings", "5a284f0bc7d30d13bc081a28")

# Load fragility mapping
fragility_service = FragilityService(client)
mapping_set = MappingSet(fragility_service.get_mapping("5b47b350337d4a3629076f2c"))
bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

In [ ]:
bldg_dmg.set_parameter("hazard_type", "earthquake")
# Memphis 7.9 AB-95, New madrid Atkinson and Boore 1995 model based hazard
bldg_dmg.set_parameter("hazard_id", "5b902cb273c3371e1236b36b")
bldg_dmg.set_parameter("num_cpu", 1)

bldg_dmg.set_parameter("result_name", "memphis_hospitals_damage")

In [ ]:
bldg_dmg.run_analysis()

In [ ]:
building_dmg_result = bldg_dmg.get_output_dataset('ds_result')

mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", building_dmg_result)
mc.set_parameter("result_name", "bldg_mc")
mc.set_parameter("num_cpu", 8)
mc.set_parameter("num_samples", 10)
mc.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3"])
mc.set_parameter("failure_state_keys", ["DS_1", "DS_2", "DS_3"])

In [ ]:
mc.run_analysis()

In [ ]:
result_fs = mc.get_output_dataset("sample_failure_state").get_dataframe_from_csv()
result_fs.head()

### Chaining with water facility damage

In [ ]:
wf_dmg = WaterFacilityDamage(client)
# Memphis water facility
wf_dmg.load_remote_input_dataset("water_facilities", "5a284f2ac7d30d13bc081e52")

# Load fragility mapping
mapping_set = MappingSet(fragility_service.get_mapping("5b47c383337d4a387669d592"))
wf_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

In [ ]:
wf_dmg.set_parameter("hazard_type", "earthquake")
# Memphis 7.9 AB-95, New madrid Atkinson and Boore 1995 model based hazard
wf_dmg.set_parameter("hazard_id", "5b902cb273c3371e1236b36b")
wf_dmg.set_parameter("fragility_key", "pga")
wf_dmg.set_parameter("use_liquefaction", True)
wf_dmg.set_parameter("liquefaction_geology_dataset_id", "5a284f53c7d30d13bc08249c")
wf_dmg.set_parameter("liquefaction_fragility_key", "pgd")
wf_dmg.set_parameter("use_hazard_uncertainty", False)
wf_dmg.set_parameter("num_cpu", 1)

wf_dmg.set_parameter("result_name", "wf_dmg_results")

In [11]:
wf_dmg.run_analysis()

True

In [12]:
water_facility_dmg_result = wf_dmg.get_output_dataset('result')

mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", water_facility_dmg_result)
mc.set_parameter("num_cpu", 8)
mc.set_parameter("num_samples", 10)
mc.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3", "DS_4"])
mc.set_parameter("failure_state_keys", ["DS_1", "DS_2", "DS_3", "DS_4"])

mc.set_parameter("result_name", "water_mc")

True

In [13]:
mc.run_analysis()

True

In [14]:
result_fs = mc.get_output_dataset("sample_failure_state")
df_fs = result_fs.get_dataframe_from_csv()
df_fs.head()

,id,failure
0,NaN,"0,0,0,0,0,0,0,0,0,0"
1,NaN,"0,0,0,0,0,0,0,0,0,0"
2,NaN,"0,0,0,0,0,0,0,0,0,0"
3,NaN,"0,0,1,0,0,0,0,0,0,0"
4,NaN,"0,0,0,0,0,0,0,0,0,0"


In [15]:
result_fp = mc.get_output_dataset("failure_probability")
df_fp = result_fp.get_dataframe_from_csv()
df_fp.head()

,LS_0,LS_1,LS_2,LS_3,DS_0,DS_1,DS_2,DS_3,DS_4,failure_probability
0,0.987977,0.912770,0.765555,0.680618,0.012023,0.075208,0.147215,0.084937,0.680618,1.0
1,0.992818,0.941305,0.826592,0.751147,0.007182,0.051512,0.114713,0.075445,0.751147,1.0
2,0.992745,0.940960,0.826160,0.751004,0.007255,0.051785,0.114800,0.075156,0.751004,1.0
3,0.991617,0.935806,0.819960,0.749020,0.008383,0.055810,0.115847,0.070939,0.749020,0.9
4,0.991851,0.936846,0.821175,0.749399,0.008149,0.055005,0.115672,0.071775,0.749399,1.0
